In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.5"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.5 Azure.Identity, 1.15.0 System.Linq.Async, 6.0.3

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.AzureAI\bin\Debug\net9.0\Microsoft.Agents.AI.AzureAI.dll"

In [5]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [6]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [7]:
using System;
using System.Linq;
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using Microsoft.Agents.AI;

In [8]:
 using DotNetEnv;

In [9]:
Env.Load("../../../.env");

In [10]:
var azure_foundry_endpoint = Environment.GetEnvironmentVariable("FOUNDRY_PROJECT_ENDPOINT") ?? throw new InvalidOperationException("AZURE_FOUNDRY_PROJECT_ENDPOINT is not set.");
var azure_foundry_model_id = Environment.GetEnvironmentVariable("FOUNDRY_MODEL_DEPLOYMENT_NAME") ?? "gpt-4.1-mini";

In [11]:
string pdfPath = "./document.md";

In [12]:
using System.IO;

async Task<Stream> OpenImageStreamAsync(string path)
{
	return await Task.Run(() => File.OpenRead(path));
}

var pdfStream = await OpenImageStreamAsync(pdfPath);

In [13]:
var persistentAgentsClient = new PersistentAgentsClient(azure_foundry_endpoint, new AzureCliCredential());

In [14]:
PersistentAgentFileInfo fileInfo = await persistentAgentsClient.Files.UploadFileAsync(pdfStream, PersistentAgentFilePurpose.Agents, "demo.md");

In [15]:
PersistentAgentsVectorStore fileStore =
            await persistentAgentsClient.VectorStores.CreateVectorStoreAsync(
                [fileInfo.Id],
                metadata: new Dictionary<string, string>() { { "agentkey", bool.TrueString } });

In [16]:
PersistentAgent agentModel = await persistentAgentsClient.Administration.CreateAgentAsync(
            azure_foundry_model_id,
            name: "DotNetRAGAgent",
            tools: [new FileSearchToolDefinition()],
            instructions: """
                You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                """,
            toolResources: new()
            {
                FileSearch = new()
                {
                    VectorStoreIds = { fileStore.Id },
                }
            },
            metadata: new Dictionary<string, string>() { { "agentkey", bool.TrueString } });

In [17]:
AIAgent agent = await persistentAgentsClient.GetAIAgentAsync(agentModel.Id);

In [18]:
AgentThread thread = agent.GetNewThread();

In [19]:
Console.WriteLine(await agent.RunAsync("Can you explain Contoso's travel insurance coverage?", thread));

Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage【4:0†demo.md】.
